# Load fit results from A1 and A2

In [75]:
all_vars = Dict();
orig_vars = Dict();
# keys = [10:13, 51:54];
keys = [10:13]
var = "phi1_orig"
orig_var = "phi0_orig"
#keys = [51:54]
#var = "phi1_log"
#orig_var = "phi0_log"
n = 100
for i = 1:n
    tmp = load("results/noisy_fits_"*string(i)*".jld", "noisy_fits")
    for key in keys
        if !haskey(all_vars, key)
            all_vars[key] = Array(Any, n);
        end
        if !haskey(orig_vars, key)
            orig_vars[key] = tmp[key][orig_var];
        end
        all_vars[key][i] = tmp[key][var]
    end
end
   

In [76]:
for key in keys
    println((sum(abs((log10(hcat(all_vars[key]...)) .- log10(orig_vars[key]))).>1,2))')
end

[74 25 4 0 2 20 39 26]
[96 0 0 0 44 71 55]
[0 0 0 27 0 27]
[0 0 0 0 0]


In [79]:
all_vars = Dict();
orig_vars = Dict();
# keys = [10:13, 51:54];
keys = [10:13]
var = "phi1_orig"
orig_var = "phi0_orig"
keys = [52:54]
var = "phi1_log"
orig_var = "phi0_log"
n = 100
for i = 1:n
    tmp = load("results/noisy_fits_"*string(i)*".jld", "noisy_fits")
    for key in keys
        if !haskey(all_vars, key)
            all_vars[key] = Array(Any, n);
        end
        if !haskey(orig_vars, key)
            orig_vars[key] = tmp[key-1][orig_var];
        end
        all_vars[key][i] = tmp[key][var]
    end
end

In [88]:
for key in keys
    println((sum(abs((log10(hcat(all_vars[key]...)) .- log10(orig_vars[key][[1,3:end]]))).>1,2))')
end

[47 2 0 0 0 9 0]
[17 63 100 100 98 0]
[100 0 0 100 100]


In [87]:
orig_vars[52][[1,3:end]]

7-element Array{Float64,1}:
  2.2e-6
  0.1026
  0.58  
  3.9e-5
  6.16  
 30.4   
  2.0   

# Generate CSV files for plotting elsewhere

In [ ]:
# Output the simulated data point markers in a CSV format with: 
# first column - voltage, second column - Popen, third column error top, fourth column

for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    println(ca_str)
    
    # Generate grid to evaluate over
    x_grid_cur = Array(Any, (1*length(V),))
    iterind = 0;
    for j1 in product([ca], V)
        iterind += 1;
        x_grid_cur[iterind] = collect(j1)
    end
    
    
    # Get the function values for original model
    tmp_vals = BK_simulator(phi0, hcat(x_grid_cur...), model_id=0)
    errbar = tmp_vals.*0.1 # Generate the error bar for every data point.
    writecsv("CSV/figure_2a_panel1_ca$(ca_str)um.csv", hcat(V[:].*1000, tmp_vals[:], errbar[:], errbar[:]))
end

# Also output the example fit with just first col - voltage, second col - Popen
# but with voltage sampled more densely to make it a continous line

Vdense = minimum(V):0.001:maximum(V) # voltage sampled every 1 millivolt

for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    println(ca_str)
    
    # Generate grid to evaluate over
    x_grid_cur = Array(Any, (1*length(Vdense),))
    iterind = 0;
    for j1 in product([ca], Vdense)
        iterind += 1;
        x_grid_cur[iterind] = collect(j1)
    end
    
    
    # Get the function values for the fit to noisy data
    tmp_vals = BK_simulator(tmp["phi1_orig"], hcat(x_grid_cur...), model_id=0)
    writecsv("CSV/figure_2a_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp_vals[:]))
end

In [ ]:
# Output the original and the fitted parameter values (first and second column of CSV respectively)
writecsv("CSV/figure_2b_panel1.csv", [phi0 tmp["phi1_orig"]])

In [4]:
# Output the statistics from all our data:
# For each model and assay
# and for each parameter 
# we will look at the standard deviation of the order of magnitude of fitted values: std(log10(fit_vals))

# First column: Original model Original Assay
# Second column: Reduced model Original Assay
# Third column: Original model Log Assay
# Fourth column: Reduced model Log Assay

writecsv("CSV/figure_A2_panel1_orig_orig.csv", std(log10(all_phi1_full_orig) ,2))
writecsv("CSV/figure_A2_panel1_red_orig.csv", std(log10(all_phi1_red_orig) ,2))
writecsv("CSV/figure_A2_panel1_orig_log.csv", std(log10(all_phi1_full_log) ,2))
writecsv("CSV/figure_A2_panel1_red_log.csv", std(log10(all_phi1_red_log) ,2))

# Collect all results into reasonable filenames

In [25]:
using JLD
phi0_full = [2.2e-6, 0.42, 0.1026, 0.58, 39*1e-6, 6.16,30.4,2.0];

n = 100
for i = 1:n
    # 10% error
    # Model 10 and 51
    res_10 = load("saves/figure_A1_hess_"*string(i)*".jld")
    res_51 = deepcopy(res_10)
    delete!(res_10, "ydata_log")
    delete!(res_10, "phi1_log")
    delete!(res_10, "hess_log")
    delete!(res_10, "D_log")
    res_10["phi0_orig"] = phi0_full;
    delete!(res_51, "ydata_orig")
    delete!(res_51, "phi1_orig")
    delete!(res_51, "hess_orig")
    delete!(res_51, "D_orig")
    res_51["phi0_log"] = phi0_full;
    
    # Model 11 and 54   
    res_11 = load("saves/figure_A2_hess_11_origmodel_54_logmodel_"*string(i)*".jld")
    res_54 = deepcopy(res_11)
    delete!(res_11, "ydata_log")
    delete!(res_11, "phi1_log")
    delete!(res_11, "hess_log")
    delete!(res_11, "D_log")
    delete!(res_11, "phi0_log")
    delete!(res_54, "ydata_orig")
    delete!(res_54, "phi1_orig")
    delete!(res_54, "hess_orig")
    delete!(res_54, "D_orig")
    delete!(res_54, "phi0_orig")
    
    # Model 12 and 52   
    res_12 = load("saves/figure_A2_hess_12_origmodel_52logmodel_"*string(i)*".jld")
    res_52 = deepcopy(res_12)
    delete!(res_12, "ydata_log")
    delete!(res_12, "phi1_log")
    delete!(res_12, "hess_log")
    delete!(res_12, "D_log")
    delete!(res_12, "phi0_log")
    delete!(res_52, "ydata_orig")
    delete!(res_52, "phi1_orig")
    delete!(res_52, "hess_orig")
    delete!(res_52, "D_orig")
    delete!(res_52, "phi0_orig")
    
    # Model 12 and 52   
    res_13 = load("saves/figure_A2_hess_"*string(i)*".jld")
    res_53 = deepcopy(res_13)
    delete!(res_13, "ydata_log")
    delete!(res_13, "phi1_log")
    delete!(res_13, "hess_log")
    delete!(res_13, "D_log")
    delete!(res_13, "phi0_log")
    delete!(res_53, "ydata_orig")
    delete!(res_53, "phi1_orig")
    delete!(res_53, "hess_orig")
    delete!(res_53, "D_orig")
    delete!(res_53, "phi0_orig")
    
    # figure_A2_hess_11_origmodel_54_logmodel_1.jld # model_id 11 and 54
    # figure_A2_hess_12_origmodel_52logmodel_1.jld  # model_id 12 and 52
    # figure_A2_hess_5.jld # model_id 13 and 53
    # figure_A1_hess_1.jld # model_id 10 and 51
    
    metadata = Dict("data_noise"=>"10%", "num_inits"=>24, "init_param_noise"=>"90%")
    
    noisy_fits = Dict(10=>res_10, 11=>res_11, 12=>res_12, 13=>res_13, 
        51=>res_51, 52=>res_52, 53=>res_53, 54=>res_54)
    
    save("results/noisy_fits_"*string(i)*".jld", "noisy_fits", noisy_fits, "metadata", metadata)
end

In [26]:
tmp = load("results/noisy_fits_1.jld")

Dict{ByteString,Any} with 2 entries:
  "metadata"   => Dict{ASCIIString,Any}("data_noise"=>"10%","init_param_noise"=…
  "noisy_fits" => Dict(54=>Dict{ByteString,Any}("ydata_log"=>[-6.38417,-5.86568…

In [33]:
load("results/noisy_fits_10.jld", "noisy_fits")[10]["phi1_orig"]

8-element Array{Float64,1}:
 1.12812e-42
 0.884739   
 0.275438   
 0.702407   
 2.95038e-5 
 1.98258e10 
 1.46832e10 
 3.29004e-10

In [29]:
tmp["noisy_fits"][11]["phi1_orig"]

7-element Array{Float64,1}:
   1.68118e-8
   0.0452016 
   0.646632  
   4.72532e-5
   2.33425   
 218.947     
   7.61379   

In [30]:
tmp["noisy_fits"][12]["phi1_orig"]

6-element Array{Float64,1}:
  0.0531189 
  0.570361  
  3.41569e-5
  0.0974806 
  2.77078   
 95.9255    

In [31]:
tmp["noisy_fits"][13]["phi1_orig"]

5-element Array{Float64,1}:
 0.0524925 
 0.543645  
 3.50317e-5
 9.42972   
 2.99854   